In [1]:
import requests
import re
import random
from bs4 import BeautifulSoup
from tqdm.auto import tqdm, trange

# Mordovians

In [2]:
url = 'https://www.mordovians.ru/mokshanskiy_yazyk'
print(url)

https://www.mordovians.ru/mokshanskiy_yazyk


In [3]:
soup = BeautifulSoup(requests.get(url).text) # forbidden, so we use html page

In [4]:
with open('sources/Мокшанский язык _ Мордовская Диаспора.html', 'r') as f:
    soup = BeautifulSoup(f.read())

In [5]:
import re

In [6]:
ru_words = []
mdf_words = []

for t in soup.find_all('div', {'class': 't-card__descr t-text t-text_sm'}):
    for item in t.find('div'):
        if item.name == 'strong':
            ru_words.append(item.text)
            mdf_words.append([])
        elif isinstance(item, str):
            item = re.sub(r'\(.*\)', ' ', item)
            text = item.strip().lstrip('–').lstrip('-').strip()
            if text:
                mdf_words[-1].append(text)

In [7]:
len(ru_words)

1784

In [8]:
all_pairs = []

for raw_ru, raw_mdf_list in zip(ru_words, mdf_words):
    if not raw_ru or not raw_mdf_list:
        continue
    ru_array = [text.strip() for text in raw_ru.split(',')]

    mdf_array = [text.strip() for raw_mdf in raw_mdf_list for text in raw_mdf.split(',') ]
    
    for ru in ru_array:
        for mdf in mdf_array:
            all_pairs.append({
                'mdf': mdf,
                'ru': ru,
                'source': url
            })

In [9]:
len(all_pairs)

1839

In [10]:
import random

In [11]:
random.choice(all_pairs)

{'mdf': 'аф оцю',
 'ru': 'небольшой',
 'source': 'https://www.mordovians.ru/mokshanskiy_yazyk'}

In [12]:
import pandas as pd

In [13]:
df = pd.DataFrame(all_pairs)

In [14]:
df.sample(10)

,mdf,ru,source
979,ваймосемс,отдыхать,https://www.mordovians.ru/mokshanskiy_yazyk
661,мянцевомс,мяться,https://www.mordovians.ru/mokshanskiy_yazyk
73,пяляскодомс,взбеситься,https://www.mordovians.ru/mokshanskiy_yazyk
1159,вешендемс,поискать,https://www.mordovians.ru/mokshanskiy_yazyk
1095,авардемс,плакать,https://www.mordovians.ru/mokshanskiy_yazyk
628,якшама,мороз,https://www.mordovians.ru/mokshanskiy_yazyk
569,тядя,мама,https://www.mordovians.ru/mokshanskiy_yazyk
1022,прважамс,отправить,https://www.mordovians.ru/mokshanskiy_yazyk
975,явоштомс,отделить,https://www.mordovians.ru/mokshanskiy_yazyk
1607,кайтан,фитиль,https://www.mordovians.ru/mokshanskiy_yazyk


In [15]:
df.to_csv("results/mosha_mordovians_dict.tsv", sep='\t')

# paevo

In [16]:
url = 'https://paevo.ru/mordva/mokshen-kyal/russko-mokshanskij-slovar'
print(url)

https://paevo.ru/mordva/mokshen-kyal/russko-mokshanskij-slovar


In [17]:
soup = BeautifulSoup(requests.get(url).text)

1. общий формат данных - **ru: mdf**
2. но бывают исключения, когда на одно русское слово несколько переводов - тогда используется ';' и скобки для уточнения контекста или наоборот, для нескольких русских слов одно мокшанское
3. поэтому будем сохранять 2 списка:
   - список, который содержит пары
   - список, которые содержит ситуации, когда на одно русское слово несколько переводов. его обработаем отдельно

In [18]:
all_pairs = []

In [19]:
special_cases = []

In [20]:
import re

table = soup.find('table')
for span in table.findAll('span'):
    for example in span.contents:
        example = str(example)
        if example == '<br/>':
            continue
        elif ':' not in example:
            # как правило сюда попадает оглавление, то есть первая буква всех русских слов ('А', 'Б', ...)
            # но также сюда попало "померить: ункстамс"
            continue

        example = re.sub(r'[\n\xa0\s]+', ' ', example).strip()
        
        if ';' in example or '(' in example or ')' in example:
            special_cases.append(example)
        elif len(example.split(':')) == 2:
            ru, mdf = example.split(':')
            mdf = re.sub(r',.*', '', mdf)  # delete plural addition

            all_pairs.append({
                'ru': ru.strip(),
                'mdf': mdf.strip(),
                'source': url
            })
        else:
            # print(example)
            special_cases.append(example)

In [21]:
len(all_pairs)

1576

In [22]:
# пример потерялся, тк в оригинале не было ':' между словом и переводом 
all_pairs.append({
    'ru': 'померить',
    'mdf': 'ункстамс',
    'source': url
})

In [23]:
len(special_cases)

68

In [24]:
for i, case in enumerate(special_cases):
    if '(' in case:
        case = re.sub('предлог', '', case)
        case = re.sub('наречие', '', case)
        case = re.sub('послелог', '', case)
        case = re.sub('наречие', '', case)
        case = re.sub('наречие', '', case)
        case = re.sub(r'\(\)', '', case)
        case = re.sub(r'\(, \)', '', case)
        special_cases[i] = case

добавим каждый особый пример индивидуально  

Для каждого слова, которое имеет несколько переводов на мордовском добавим оба этих перевода. А если пояснен контекст, то и для этого контекста добавим это слово. То есть для "взгляд: ванфкс, -т; (мнение): мяль, -льхть" получим 3 пары: 'взгляд: ванфкс', 'взгляд: мяль', 'мнение: мяль'

Но это правило не всегда будет работать. Для "вишня (дерево): атямарькс, -т; (плод): атямарь, -рьхть" добавим 2 примера: 'вишня (дерево): атямарькс', 'вишня (плод): атямарь'

In [25]:
# with open("sources/paevo_special_cases.txt", 'w') as file:
#     file.write('\n'.join(special_cases))

In [26]:
# cчитаем после обработки
with open("sources/paevo_special_cases.txt", 'r') as file:
    special_cases = file.read().split('\n')


In [27]:
len(special_cases)

144

In [28]:
for example in special_cases:
    ru, mdf = example.split(':')

    all_pairs.append({
        'ru': ru.strip(),
        'mdf': mdf.strip(),
        'source': url
    })

In [29]:
len(all_pairs)

1721

In [30]:
df = pd.DataFrame(all_pairs)

In [31]:
df.sample(10)

,ru,mdf,source
1673,лопнуть,лопадемс,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
1253,рябина,пизелкс,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
1459,узнать,содамс,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
481,отвезти,ускомс,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
132,голубь,гуля,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
1702,произнести имя,азомс,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
873,играть,налхкомс,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
965,кролик,куднумол,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
513,отложить,путомс бокс,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
1446,убрать,урядамс,https://paevo.ru/mordva/mokshen-kyal/russko-mo...


In [32]:
df.to_csv("results/paevo_dict.tsv", sep='\t')

# Найденные в интернете отдельные слова

1. https://rousseau.livejournal.com/175111.html
2. https://ru.wiktionary.org/wiki/%D0%9F%D1%80%D0%B8%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5:%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%A1%D0%B2%D0%BE%D0%B4%D0%B5%D1%88%D0%B0_%D0%B4%D0%BB%D1%8F_%D0%BC%D0%BE%D0%BA%D1%88%D0%B0%D0%BD%D1%81%D0%BA%D0%BE%D0%B3%D0%BE_%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0



In [33]:
with open("sources/extra_words.txt", 'r') as file:
    extra_words = file.read().split('\n')

In [34]:
len(extra_words)

244

In [35]:
all_pairs = []
for example in extra_words:
    ru, mdf = example.split(':')

    all_pairs.append({
        'ru': ru.strip(),
        'mdf': mdf.strip(),
        'source': 'extra_words'
    })

In [36]:
len(all_pairs)

244

In [37]:
df = pd.DataFrame(all_pairs)

In [38]:
df.sample(10)

,ru,mdf,source
134,сялгомс,проколоть,extra_words
83,сюра,рог,extra_words
226,покаряв,круглый,extra_words
218,пара,добрый,extra_words
108,ярхцамс,есть,extra_words
27,изь,не,extra_words
127,кунцемс,ловить,extra_words
221,кальдяв,плохой,extra_words
109,сускомс,кусать,extra_words
33,вейке,один,extra_words


In [39]:
df.to_csv("results/extra_words.tsv", sep='\t')

# Merge all dicts

In [40]:
import pandas as pd

In [41]:
df_dicts = pd.concat([
    pd.read_csv('results/paevo_dict.tsv', sep='\t'),
    pd.read_csv('results/mosha_mordovians_dict.tsv', sep='\t'),
    pd.read_csv('results/extra_words.tsv', sep='\t'),
    pd.read_csv('results/book_dicts.tsv', sep='\t'),
    pd.read_csv('results/book_words.tsv', sep='\t')
])[['ru', 'mdf', 'source']].reset_index(drop=True)

In [42]:
df_dicts.sample(10)

,ru,mdf,source
1021,ломтик,печфкя,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
6093,попасть,повомс,mdf-ru-dict
6709,греть,эждемс,book
2523,ножницы,васоньбеельхть,https://www.mordovians.ru/mokshanskiy_yazyk
4439,мычать,парамс,ru-mdf-dict
5688,голубь,гблый,mdf-ru-dict
1278,свистеть,вяшкомс,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
3236,считать,лувомс,https://www.mordovians.ru/mokshanskiy_yazyk
2160,колено,плманжа,https://www.mordovians.ru/mokshanskiy_yazyk
1764,бородавка,цильге,https://www.mordovians.ru/mokshanskiy_yazyk


In [43]:
df_dicts = df_dicts.dropna()

In [44]:
df_dicts['ru'] = df_dicts['ru'].apply(lambda x: x.lower().strip('.,!? '))

In [45]:
df_dicts['mdf'] = df_dicts['mdf'].apply(lambda x: x.lower().strip('.,!? '))

In [46]:
df_dicts = df_dicts.drop_duplicates(subset=['ru', 'mdf']).reset_index(drop=True)

In [47]:
df_dicts.shape

(3589, 3)

In [48]:
df_dicts = df_dicts.dropna()

In [49]:
df_dicts.shape

(3589, 3)

In [63]:
df_dicts.sample(10)

,ru,mdf,source
3533,пресный,салфтома,book
3474,валенки,валенцят,book
911,кислый,шапама,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
1865,шелуха,кедь,https://www.mordovians.ru/mokshanskiy_yazyk
647,подсолнечник,шинжарома,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
182,жеребенок,вашеня,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
3350,вчера,исяк,book
2932,работать,покамс,mdf-ru-dict
1146,несколько,мзярошкавок,https://paevo.ru/mordva/mokshen-kyal/russko-mo...
2604,застолье по поводу,ила,mdf-ru-dict


In [64]:
df_dicts.to_csv('results/all_dicts_data.tsv', sep='\t')